- https://www.six-sigma-material.com/Proportions-Tests.html
- https://online.stat.psu.edu/statprogram/reviews/statistical-concepts/proportions
- each pValue < 0.01 meaning we are even 99% sure that null hypo must not be true
- I have tried all methods but all reject null hypo

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

def pTester(ratio_1,ratio_2,totalPop):
    p0=(ratio_2+ratio_1)/2 # under null hypo
    zStat=(ratio_1-p0)/np.sqrt(p0*(1-p0)/totalPop)
    pValue=stats.distributions.norm.cdf(-np.abs(zStat))
    return pValue

def pTester(ratio_1,ratio_2,totalPop):
    p0=0.5 # under null hypo
    zStat=(ratio_1-p0)/np.sqrt(p0*(1-p0)/totalPop)
    pValue=stats.distributions.norm.cdf(-np.abs(zStat))
    return pValue

#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePopMale=census.iloc[stateCode,6]
    statePopFemale=census.iloc[stateCode,7] 
    # modify state code
    stateCode=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no 
    # note: these col numbers are specific to df #
    male=sum(df.iloc[:,15].tolist())
    female=sum(df.iloc[:,16].tolist())
    #========
    # should I use statePops or (male+female) for 3+ langs to devide
    # right now I am doing thid with male+female for 3+ langs
    # in fact I feel why I am including whole state pops to devide my study pop that is persons 3+
    #========
    totalPop=male+female
    #totalPop=statePopFemale+statePopMale
    ratio_1=(male/totalPop) # no rounding off
    ratio_2=(female/totalPop)
    #pValue=pTester(ratio_1,ratio_2,totalPop) # func that gives pValue for ratios
    pValue=pTester(male,female,totalPop)
    #store ratios in a dict
    stateRatios={
        'StateCode':stateCode,
        'male-percentage':ratio_1,
        'female-percentage':ratio_2,
        'p-value':pValue
    }
    #print('Processed state -- ',stateCode)
    return stateRatios

#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================
####based on: https://online.stat.psu.edu/stat415/lesson/9/9.4####
def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePopMale=census.iloc[stateCode,6]
    statePopFemale=census.iloc[stateCode,7]
    # modify state code
    stateCode=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no
    # note: these col numbers are specific to df #
    male=sum(df.iloc[:,15].tolist())
    female=sum(df.iloc[:,16].tolist())
    #========
    # should I use statePops or (male+female) for 3+ langs to devide
    # right now I am doing thid with male+female for 3+ langs
    # in fact I feel why I am including whole state pops to devide my study pop that is persons 3+
    #========
    totalPop=male+female
    #totalPop=statePopFemale+statePopMale
    ratio_1=(male/statePopMale) # no rounding off
    ratio_2=(female/statePopFemale)
    #pValue=pTester(ratio_1,ratio_2,totalPop) # func that gives pValue for ratios
    #pValue=pTester(male,female,totalPop)
    ratio=totalPop/(statePopFemale+statePopMale)
    zStat=(ratio_1-ratio_2)/(np.sqrt((ratio*(1-ratio))*((1/statePopFemale)+(1/statePopMale))))
    pValue=2*stats.distributions.norm.cdf(-np.abs(zStat))
    #store ratios in a dict
    stateRatios={
        'StateCode':stateCode,
        'male-percentage':ratio_1,
        'female-percentage':ratio_2,
        'p-value':pValue
    }
    #print('Processed state -- ',stateCode)
    return stateRatios

In [13]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================
####based on chi quare test with df=1
def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePopMale=census.iloc[stateCode,6]
    statePopFemale=census.iloc[stateCode,7]
    # modify state code
    stateCode=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no
    # note: these col numbers are specific to df #
    male=sum(df.iloc[:,15].tolist())
    female=sum(df.iloc[:,16].tolist())
    #========
    # should I use statePops or (male+female) for 3+ langs to devide
    # right now I am doing thid with male+female for 3+ langs
    # in fact I feel why I am including whole state pops to devide my study pop that is persons 3+
    #========
    totalPop=male+female
    #totalPop=statePopFemale+statePopMale
    ratio_1=(male/statePopMale) # no rounding off
    ratio_2=(female/statePopFemale)
    #pValue=pTester(ratio_1,ratio_2,totalPop) # func that gives pValue for ratios
    #pValue=pTester(male,female,totalPop)
    ratio=(ratio_1+ratio_2)/2
    zStat=((ratio_1-ratio)**2+(ratio_2-ratio)**2)/ratio
    pValue=1-stats.distributions.chi2.sf(np.abs(zStat),1)
    #store ratios in a dict
    stateRatios={
        'state-code':stateCode,
        'male-percentage':ratio_1,
        'female-percentage':ratio_2,
        'p-value':pValue
    }
    #print('Processed state -- ',stateCode)
    return stateRatios

In [14]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)

CPU times: user 19.4 s, sys: 16.1 ms, total: 19.4 s
Wall time: 19.4 s


In [15]:
ratioDF
#based on chi-square
# but it is not for this

,StateCode,male-percentage,female-percentage,p-value
0,00,0.081083,0.060370,0.043919
1,01,0.189553,0.141928,0.065925
2,02,0.057578,0.043399,0.035589
3,03,0.299454,0.262981,0.038788
4,04,0.307505,0.302077,0.005548
5,05,0.021318,0.015822,0.022755
6,06,0.050481,0.041271,0.024255
7,07,0.082507,0.079060,0.006840
8,08,0.018082,0.011028,0.032977
9,09,0.014807,0.010895,0.019464
